In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
dfGames= pd.read_parquet("./lichess-sampled.parquet")
#dfGames = dfGames.reset_index()
dfGames.head(3)

,Event,Site,White,Black,Result,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,TimeControl,Termination,Moves,WhiteTitle,BlackTitle,LichessId,Date
0,Classical,https://lichess.org/a9tcp02g,Desmond_Wilson,savinka59,White,1654.0,1919.0,19.0,-22.0,D04,480+2,Normal,1. d4 d5 2. Nf3 Nf6 3. e3 Bf5 4. Nh4 Bg6 5. Nx...,NaN,NaN,<NA>,2012-12-31 23:04:12
1,Classical,https://lichess.org/iclkx584,Voltvolf,Marzinkus,White,1824.0,1811.0,11.0,-11.0,C02,360+6,Normal,1. e4 e6 2. d4 d5 3. e5 c5 4. c3 Ne7 5. f4 cxd...,NaN,NaN,<NA>,2012-12-31 23:10:00
2,Classical,https://lichess.org/ufcqmfxx,6WX,adamsrj,White,1463.0,1504.0,62.0,-12.0,C44,1560+30,Normal,1. e4 e5 2. Nf3 Nc6 3. Bc4 Be7 4. d4 exd4 5. N...,NaN,NaN,<NA>,2012-12-31 23:16:04


In [3]:
openings = {}
with open('eco-codes.txt') as f:
    lines = f.readlines() # list containing lines of file
    
    for i, line in enumerate(lines):
        lineArray = line.split(": ")
        openings[i] = [lineArray[0], lineArray[1][:-1]]
        #openings[lineArray[0]] = lineArray[1][:-1]
        
dfOpenings = pd.DataFrame(data=openings, index=["Code", "Opening Name"]).T
#dfOpenings = pd.DataFrame(data=openings, index=["Opening Name"]).T
dfOpenings.loc["?"] = "Unknown"
dfOpenings.head(3)

,Code,Opening Name
0,A00,Irregular Openings
1,A01,Larsen's Opening
2,A02,Bird's Opening


In [4]:
dfGames = pd.merge(dfGames, dfOpenings, left_on="ECO", right_on="Code").drop("Code", axis=1)
dfGames = dfGames.reindex(columns=["Event", "Site", "White", "Black", "Result", "WhiteElo",
                                   "BlackElo", "WhiteRatingDiff", "BlackRatingDiff", "ECO", "Opening Name",
                                   "TimeControl", "Termination", "Moves", "WhiteTitle", "BlackTitle",
                                   "LichessId", "Date"])
dfGames["Opening Name"] = dfGames["Opening Name"].astype("category")
dfGames.head(3)

,Event,Site,White,Black,Result,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening Name,TimeControl,Termination,Moves,WhiteTitle,BlackTitle,LichessId,Date
0,Classical,https://lichess.org/a9tcp02g,Desmond_Wilson,savinka59,White,1654.0,1919.0,19.0,-22.0,D04,Queen's Pawn Game,480+2,Normal,1. d4 d5 2. Nf3 Nf6 3. e3 Bf5 4. Nh4 Bg6 5. Nx...,NaN,NaN,<NA>,2012-12-31 23:04:12
1,Classical,https://lichess.org/ufwrwgqn,Malkav,bluemoon77,White,1298.0,1261.0,12.0,-11.0,D04,Queen's Pawn Game,300+5,Normal,1. d4 Nf6 2. e3 d5 3. Nf3 Bg4 4. h3 Bh5 5. Be2...,NaN,NaN,<NA>,2013-01-16 14:17:09
2,Classical,https://lichess.org/gjp68y4l,Nespro,FGR,Draw,1678.0,1456.0,-5.0,10.0,D04,Queen's Pawn Game,840+8,Normal,1. d4 d5 2. Nf3 Nf6 3. e3 Nc6 4. Bb5 a6 5. Bxc...,NaN,NaN,<NA>,2013-01-17 19:04:53


In [12]:
gp = dfGames.groupby(["Opening Name"])
gp.describe()

WhiteElo               \
                                                  count         mean   
Opening Name                                                           
Alekhine's defense                              41200.0  1480.111189   
Alekhine's defense, Modern variation              886.0  1734.133183   
Alekhine's defense, Modern variation, 4...Bg4     438.0  1792.627854   
Benko gambit                                     2458.0  1806.826282   
Benko gambit accepted                             631.0  1939.860539   
...                                                 ...          ...   
Torre Attack, Tartakower variation               2455.0  1638.566191   
Two Knights Defense                            102762.0  1581.335153   
Vienna Gambit, Kaufmann Variation                 401.0  1748.645885   
Vienna Game                                     33462.0  1478.587174   
Vienna Game, Falkbeer Variation                 22261.0  1513.911145   

                                                                            \
                                                      std     min      25%   
Opening Name                                                                 
Alekhine's defense                             300.147183   612.0  1251.00   
Alekhine's defense, Modern variation           265.177238   930.0  1559.25   
Alekhine's defense, Modern variation, 4...Bg4  238.534726  1146.0  1637.00   
Benko gambit                                   228.420854   977.0  1665.25   
Benko gambit accepted                          182.975971  1258.0  1823.00   
...                                                   ...     ...      ...   
Torre Attack, Tartakower variation             269.047175   770.0  1463.00   
Two Knights Defense                            220.644688   600.0  1439.00   
Vienna Gambit, Kaufmann Variation              285.553024   923.0  1555.00   
Vienna Game                                    274.526292   635.0  1280.00   
Vienna Game, Falkbeer Variation                247.604092   773.0  1347.00   

                                                                       \
                                                  50%     75%     max   
Opening Name                                                            
Alekhine's defense                             1473.0  1699.0  2685.0   
Alekhine's defense, Modern variation           1742.0  1911.5  2568.0   
Alekhine's defense, Modern variation, 4...Bg4  1799.5  1978.5  2514.0   
Benko gambit                                   1821.5  1959.0  2475.0   
Benko gambit accepted                          1947.0  2069.5  2455.0   
...                                               ...     ...     ...   
Torre Attack, Tartakower variation             1646.0  1834.5  2460.0   
Two Knights Defense                            1581.0  1727.0  2627.0   
Vienna Gambit, Kaufmann Variation              1768.0  1941.0  2498.0   
Vienna Game                                    1480.0  1666.0  2528.0   
Vienna Game, Falkbeer Variation                1513.0  1686.0  2564.0   

                                               BlackElo               ...  \
                                                  count         mean  ...   
Opening Name                                                          ...   
Alekhine's defense                              41200.0  1465.599442  ...   
Alekhine's defense, Modern variation              886.0  1733.067720  ...   
Alekhine's defense, Modern variation, 4...Bg4     438.0  1822.958904  ...   
Benko gambit                                     2458.0  1823.564687  ...   
Benko gambit accepted                             632.0  1946.245253  ...   
...                                                 ...          ...  ...   
Torre Attack, Tartakower variation               2455.0  1650.601629  ...   
Two Knights Defense                            102764.0  1565.781567  ...   
Vienna Gambit, Kaufmann Variation                 401.0  1723.029925  ...  